# Assignment for Topic 5

## Part 1

Analyse the differences between the sexes by age in Ireland (not regions)

Using CSO data ((https://data.cso.ie/), load data from the FY006A - Population database. 

### Import the csv data from the url containing the data

In [ ]:
# import pandas

import pandas as pd


In [ ]:
# Define url
url = https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/JSON-stat/2.0/en

# Define dataframe and import data from url 
df = pd.read_csv(url)

# Sanity check, show first 5 rows
df.head(5)

# check this against csv i downloaded from the CSO (to be sure)
